<a href="https://colab.research.google.com/github/joekakone/learn_data_science/blob/master/bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 2: Apprentissage supervisé

* 2.1: Les K plus Proches Voisins [[Notebook]](2_1_knn.ipynb)
* 2.2: Régression linéaire [[Notebook]](2_2_régression_linéaire.ipynb)
* 2.3: Régression logistique [[Notebook]](2_3_régression_logistique.ipynb)
* 2.4: Séparatrices à Vastes Marges [[Notebook]](2_4_svm.ipynb)
* 2.5: Arbres de décision [[Notebook]](2_5_trees.ipynb)
* **2.6: Bagging** [[Notebook]](2_6_bagging.ipynb)
* 2.7: Forêts Aléatoires [[Notebook]](2_7_random_forest.ipynb)
* 2.8: Gradient Boosting [[Notebook]](2_8_gradient_boosting.ipynb)

# Objectif
Notre objectif est de prédire si un patient souffre ou non du diabète. Il s'agit d'un problème d'apprentissage supervisé, problème de classification.

# Importer les libairies nécessaires

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# Importer les données depuis un fichier
La commande pour importer des données depuis un fichier `.csv` c'est `pandas.read_csv`.

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/joekakone/datasets/master/datasets/datasets_for_learn_data_science/datasets_228_482_diabetes.csv')
data.head()

# Afficher les dimensions du tableau

In [ ]:
data.shape

Le jeu de données contient 768 données et 9 variables, les huit variables explicatives et une variable expliquée `Outcome`

In [ ]:
# Nombre de variables explicatives
n_features = data.shape[1] - 1
# Liste des variales explicatives
features = data.columns[:-1]
n_features

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data[data.isna()].count()

# Variable expliquée

La variable expliquée est `Outcome`. Elle est encodée en 0 et 1, 0 pour négatif et 1 pour positif.

## Table des effectifs
La méthode `.value_counts`

In [ ]:
Outcome = data['Outcome'].value_counts(sort=False)

Outcome

Deux types de graphes sont adaptés pour représenter: le diagramme en bâtons et le diagramme en secteurs.

Dans la pratique, lorsque le nombre de modalités dépasse trois, le diagramme en bâtons est préféré au digramme en secteurs. La comparaison est plus facile. Les tuyaux d'orgue sont une variante du diagramme en bâtons.

## Table des fréquences
Nous utilisons toujours la méthode `.value_counts` mais cette fois-ci en donnant la valeur `True` à l'argument `normalize` .

In [ ]:
Outcome = data['Outcome'].value_counts(sort=False, normalize=True)
Outcome

## Diagramme en bâtons

In [ ]:
# diagramme en bâtons
Outcome.plot(kind='bar')

## Diagramme en secteurs (camenbert)

In [ ]:
# diagramme en secteurs
Outcome.plot(kind='pie')

Le jeu de données est déséquilibré. Il y'a plus de `65%` de données négatives contre un peu plus de `34%` de données négatives. Il faudra en tenir compte de ce déséquilibre lors de l'échantillonnage des données d'entraînement.

# Variables explicatives

In [ ]:
# les boîtes à moustaches
sns.boxplot(data=data.drop(['Outcome'], axis=1))

En dessinant les boîtes à moustaches comme ci-dessus, on constate qu'il y'a une différence d'echelle entre les variables. Et on peut commencer par envisager de les centrer et de les réduire pour les ramener à la même échelle.

On remarque par ailleurs la présence de plusieurs valeurs atypiques pour la variable `Insulin`. Nous allons regarder de plus près en étudiant les variables une à une.

In [ ]:
fig = plt.figure(figsize=(12, 20))

for i in range(n_features):
    fig.add_subplot(4, 2, i+1)
    sns.boxplot(x=data[features[i]])
#     plt.xlabel('')
#     plt.title(features[i])
    
plt.show()

In [ ]:
sns.distplot(data['Insulin'])
plt.show()

L'histogramme 
Nous allons enlever les données pour lesquelles `Insulin > 400`.

In [ ]:
sns.distplot(data['DiabetesPedigreeFunction'])
plt.show()

De même que pour la variable `Insulin`, nous allons enlever les données pour lesquelles `DiabetesPedigreeFunction > 1.5`.

## Suppression des données atypiques (outliers)

In [ ]:
data = data[data['Insulin']<=400]
data = data[data['DiabetesPedigreeFunction']<=1.5]

data.shape

Nous avons ainsi enlevé `768-740=28` données atypiques qui sont succeptible de biaisé la modélisation.

In [ ]:
fig = plt.figure(figsize=(12, 4))

fig.add_subplot(1, 2, 1)
sns.distplot(data['Insulin'])

fig.add_subplot(1, 2, 2)
sns.distplot(data['DiabetesPedigreeFunction'])

plt.show()

In [ ]:
Pregnancies = data['Pregnancies'].value_counts(sort=False)
Pregnancies

In [ ]:
Pregnancies.plot(kind='bar')

In [ ]:
sns.pairplot(data=data, hue='Outcome')

In [ ]:
corr = data.drop(['Outcome'], axis=1).corr()
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values)
plt.show()

# ACP - Analyse en Composantes Principales
L'ACP est une technique de réduction de dimaensions.

Nous allons ulitiser l'ACP pour réduire le nombre de dimensions et ainsi visualiser les données en deux dimensions. Nous pourrons ainsi juger les facteurs discriminants de nos variables explicatives.

## Centrage et Réduction

In [ ]:
from sklearn.preprocessing import StandardScaler

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)

X_projected = pca.fit_transform(X_scaled)

In [ ]:
fig = plt.figure(figsize=(12, 4))

fig.add_subplot(1, 2, 1)
plt.scatter(X_projected[:, 0], X_projected[:, 1], c=y)
plt.xlabel('F1')
plt.ylabel('F2')
plt.title('Premier plan factoriel')

fig.add_subplot(1, 2, 2)
plt.scatter(X_projected[:, 2], X_projected[:, 3], c=y)
plt.xlabel('F3')
plt.ylabel('F4')
plt.title('Deuxième plan factoriel')

plt.show()

In [ ]:
for i in range(n_features):
    ft = data[features[i]]
#     print(features[i], ft.min(), ft.mean(), ft.max())
    print((ft.max() - ft.min())/10)

# Modélisation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [ ]:
X = data.iloc[:, :-1] # variables
y = data.iloc[:, -1] # étiquettes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2) # 80% pour le jeu d'entraînement

## Bagging

Le Bagging est méthode ensembliste qui consiste à entraîner plusieurs modèles de manière parallèle.

Le Bootstraping est une technique d'échantillonnage utilisée en Statistique. Elle consiste à créer des sous-echantillons de même taille que que l'echantillon de travail. On fait des tirages avec remise.

In [ ]:
# bg = BaggingClassifier(n_estimators=20, bootstrap_features=True)
bg = BaggingClassifier(n_estimators=20) # 20 modèles

In [ ]:
bg.fit(X_train, y_train)

bg.score(X_train, y_train)

# Évaluatio

In [ ]:
y_pred = bg.predict(X_test)

acc = accuracy_score(y_test, y_pred) 
f1 = f1_score(y_test, y_pred)
confmat = confusion_matrix(y_test, y_pred)

print('Accuracy: %0.2f' % acc)
print('F1 score: %0.2f' % f1)

plt.matshow(confmat, cmap=plt.cm.Greens, alpha=.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        plt.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
plt.xlabel('Valeur prédite')
plt.ylabel('Vraie valeur')
plt.show()